In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

In [ ]:
!unzip asl-alphabet.zip

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

In [ ]:
for c in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
  os.mkdir('/content/asl_alphabet_test/asl_alphabet_test/'+c+'/')

os.mkdir('/content/asl_alphabet_test/asl_alphabet_test/'+'del'+'/')
os.mkdir('/content/asl_alphabet_test/asl_alphabet_test/'+'nothing'+'/')
os.mkdir('/content/asl_alphabet_test/asl_alphabet_test/'+'space'+'/')

for c in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
  os.replace('/content/asl_alphabet_test/asl_alphabet_test/'+c+'_test.jpg', '/content/asl_alphabet_test/asl_alphabet_test/'+c+'/'+c+'.jpg')

os.replace('/content/asl_alphabet_test/asl_alphabet_test/nothing_test.jpg', '/content/asl_alphabet_test/asl_alphabet_test/nothing/nothing.jpg')
os.replace('/content/asl_alphabet_test/asl_alphabet_test/space_test.jpg', '/content/asl_alphabet_test/asl_alphabet_test/space/space.jpg')

In [ ]:
checkpoint_path = "/content/training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
import pathlib
train_data_dir = pathlib.Path('/content/asl_alphabet_train/asl_alphabet_train/')
test_data_dir = pathlib.Path('/content/asl_alphabet_test/asl_alphabet_test')

In [ ]:
image_count = len(list(train_data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
train_data_dir

In [ ]:
l = list(train_data_dir.glob('A/*'))

In [ ]:
# l

In [ ]:
A = PIL.Image.open(l[0])
width, height = A.size

In [ ]:
A

In [ ]:
width

In [ ]:
height

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_data_dir,
  image_size=(height, width),
  batch_size=32)


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_data_dir,
  image_size=(height, width),
  batch_size=32)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
'''
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
'''

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
num_classes = 29

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(height, width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
import pickle

epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[cp_callback]
)
with open('/trainHistoryDict', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(val_ds)

In [ ]:
model.save('/content/saved_model/')

In [ ]:
print(history.history.keys())

In [ ]:
acc_train = history.history['accuracy']
acc_val = history.history['val_accuracy']
epochs = range(1,11)
plt.plot(epochs, acc_train, 'g', label='Training accuracy')
plt.plot(epochs, acc_val, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig("Model_Accuracy.pdf")
files.download("Model_Accuracy.pdf")
plt.show()

In [ ]:
import pickle
with open('trainHistoryDict', 'rb') as file:
  x = pickle.load(file)

In [ ]:
import matplotlib.pyplot as plt
acc_train = x['accuracy']
acc_val = x['val_accuracy']
epochs = range(1,11)
plt.plot(epochs, acc_train, 'g', label='Training accuracy')
plt.plot(epochs, acc_val, 'b', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
# plt.savefig("Model_Accuracy.pdf")
# files.download("Model_Accuracy.pdf")
plt.show()